In [1]:
# step07 : 역전파 자동화
import numpy as np

class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None  
    self.creator = None # 추가 (함수 저장)

  def set_creator(self, func):  # 추가
    self.creator = func

  def backward(self): # 추가
    f = self.creator  # 1. 함수를 가져옴
    if f is not None:
      x = f.input # 2. 함수 입력을 가져옴
      x.grad = f.backward(self.grad)  # 3. backward 호출
      x.backward()  # 하나 앞 변수의 backward 호출 (재귀))


class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    output.set_creator(self)  # 출력 변수에 창조자 설정
    self.input = input
    self.output = output  # 출력도 저장
    return output
  
  def forward(self, x):
    raise NotImplementedError()
  
  def backward(self, x):
    raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# backward
y.grad = np.array(1.0)
y.backward()
print(x.grad)


3.297442541400256


In [3]:
#step08 : 재귀에서 반복문으로

class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None  
    self.creator = None 

  def set_creator(self, func):
    self.creator = func

  def backward(self): 
    funcs = [self.creator]  # funcs 리스트 생성
    while funcs:
      f = funcs.pop() # 함수를 가져옴
      x, y = f.input, f.output  # 함수의 입출력 가져옴
      x.grad = f.backward(y.grad)  # backward 호출
      
      if x.creator is not None:
        funcs.append(x.creator)  # 하나 앞의 함수를 추가

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# backward
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [4]:
#step09 : 함수를 더 편리하게
class Variable:
  def __init__(self, data):
    # np.array(1.0) or None 제외하고는 오류 발생하게
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은 지원하지 않습니다'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None

    def set_creator(self, func):
      self.creator = func

    def backward(self):
      # np.array(1.0) 대신 None으로 작성 가능하게
      if self.grad is None:
        self.grad = np.ones_like(self.data)

      funcs = [self.creator]
      while funcs:
        f = funcs.pop()
        x, y = f.input, f.output
        x.grad = f.backward(y.grad)

        if x.creator is not None:
          funcs.append(x.creator)


# 0차원 입력(scalar) 들어오면 1차원으로 변환 
def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x


class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(as_array(y))
    output.set_creator(self)
    self.input = input
    self.output = output
    return output

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()


class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y

  def backward(self, gy):
    x = self.input.data
    gx = 2 * x * gy
    return gx


class Exp(Function):
  def forward(self, x):
    y = np.exp(x)
    return y

  def backward(self, gy):
    x = self.input.data
    gx = np.exp(x) * gy
    return gx


def square(x):
  return Square()(x)  # 한 줄로 작성


def exp(x):
  return Exp()(x)


x = Variable(np.array(0.5))
y = square(exp(square(x)))  # 연속 적용
y.backward()
print(x.grad)


x = Variable(np.array(1.0))  # OK
x = Variable(None)  # OK
x = Variable(1.0)  # NG

AttributeError: 'Variable' object has no attribute 'set_creator'